In [1]:
import numpy as np
import jax
import jax.numpy as jnp
import json
import pickle

import scipy
from scipy import constants
from scipy import stats
import astropy.constants


from wtf import wtf_compile

2025-09-05 17:10:04.114254: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-09-05 17:10:04.125653: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-09-05 17:10:04.129182: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-09-05 17:10:04.771865: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
with open('pitchfork/pitchfork.json', 'r') as fp:
    pitchfork_dict = json.load(fp)

pitchfork = wtf_compile(pitchfork_dict, jaxxed=False)

with open('pitchfork/pitchfork_info.json', 'r') as fp:
    pitchfork_info = json.load(fp)

pitchfork_cov = np.loadtxt('pitchfork/pitchfork_covariance.txt')

In [3]:
pca_comps = np.array(pitchfork_info['custom_objects']['inverse_pca']['pca_comps'])
pca_mean = np.array(pitchfork_info['custom_objects']['inverse_pca']['pca_mean'])


def pcann(x, jaxxed=False):
    if jaxxed:
        y = jnp.tensordot(x, pca_comps, 1) + pca_mean
        return y
    else:
        y = np.tensordot(x, pca_comps, 1) + pca_mean
        return y



In [6]:
preds = pitchfork.forward(np.array([[0.5,0.5,0.5,0.5,0.5]]))
preds

[array([[ 2.16970188e+00, -2.69331376e-02, -2.68745388e-02,
         -5.39015889e-03,  4.58675991e-03, -1.58866712e-03,
          3.01504437e-03,  1.99272068e-03, -1.26795136e-04,
          1.98271638e-03,  1.32488493e-03,  1.57268512e-03,
          2.23839220e-05,  5.42823507e-04, -3.67025738e-03]]),
 array([[5.34786024, 3.61132239, 1.03881122]])]

In [65]:
def pcann(x, jaxxed=False):
    if jaxxed:
        y = jnp.tensordot(x, pca_comps, 1) + pca_mean
        return y
    else:
        y = np.tensordot(x, pca_comps, 1) + pca_mean
        return y


def predict(x, emulator, jaxxed=False):
    if jaxxed:
        pass
    else:
        log_inputs_mean = np.array(pitchfork_info["data_scaling"]["inp_mean"][0])
        
        log_inputs_std = np.array(pitchfork_info["data_scaling"]["inp_std"][0])

        log_outputs_mean = np.array(pitchfork_info["data_scaling"]["classical_out_mean"][0] + pitchfork_info["data_scaling"]["astero_out_mean"][0])
        
        log_outputs_std = np.array(pitchfork_info["data_scaling"]["classical_out_std"][0] + pitchfork_info["data_scaling"]["astero_out_std"][0])
            
        log_inputs = np.log10(x)
        
        standardised_log_inputs = (log_inputs - log_inputs_mean)/log_inputs_std

        preds = pitchfork.forward(standardised_log_inputs)
        
        standardised_log_outputs = np.concatenate((np.array(preds[1]),np.array(pcann(preds[0]))), axis=1)

        log_outputs = (standardised_log_outputs*log_outputs_std) + log_outputs_mean

        outputs = 10**log_outputs

        outputs[:,2] = log_outputs[:,2] ##we want star_feh in dex

        # teff = np.array(((outputs[:,1]*astropy.constants.L_sun) / (4*np.pi*constants.sigma*((outputs[:,0]*astropy.constants.R_sun)**2)))**0.25)
        
        # outputs[:,0] = teff
        
        # #outputs = np.concatenate((np.array(outputs[:,:3]), np.array(outputs[:,n_min-3:n_max-2])), axis=1)

        return outputs
        

In [66]:
predict(np.array([[0.5,0.5,0.5,0.5,0.5]]), pitchfork)

array([[1.69203956e+28, 2.58278194e+36, 3.66121808e+01, 7.98343758e-04,
        8.27478264e+01, 2.98534905e+01, 3.09840047e+03, 1.69057150e+04,
        3.39187159e+01, 2.46668441e-01, 1.94838973e+00, 1.01935737e+02,
        1.47973240e+02, 2.33913783e+01, 1.82341612e+01, 1.44833682e+02,
        1.67798033e+03, 5.34928744e+03, 4.20138033e+03, 2.08572397e+03,
        1.31815286e+03, 1.47132625e+03, 3.22099318e+03, 7.15133084e+03,
        5.22700231e+03, 1.90555167e+03, 2.27872553e+03, 1.96758277e+04,
        2.44156811e+05, 3.89735928e+05, 3.77836413e+04, 1.33173966e+03,
        1.91749251e+02, 2.62525315e+02, 1.01727011e+03, 2.32002517e+03,
        2.24504708e+03, 1.74631346e+03]])

In [ ]:
class InversePCA(tf.keras.layers.Layer):
    """
    Inverse PCA layer for tensorflow neural network
    
    Usage:
        - Define dictionary of custom objects containing Inverse PCA
        - Use arguments of PCA mean and components from PCA of output parameters for inverse PCA (found in JSON dict)
        
    Example:

    > f = open("pcann_info.json")
    >
    > data = json.load(f)
    >
    > pca_comps = np.array(data["pca_comps"])
    > pca_mean = np.array(data["pca_mean"])
    > 
    > custom_objects = {"InversePCA": InversePCA(pca_comps, pca_mean)}
    > pcann_model = tf.keras.models.load_model("pcann_name.h5", custom_objects=custom_objects)
    
    """
    
    def __init__(self, pca_comps, pca_mean, **kwargs):
        super(InversePCA, self).__init__()
        self.pca_comps = pca_comps
        self.pca_mean = pca_mean
        
    def call(self, x):
        y = tf.tensordot(x, np.float32(self.pca_comps),1) + np.float32(self.pca_mean)
        return y
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'pca_comps': self.pca_comps,
            'pca_mean': self.pca_mean
        })
        return config

class WMSE(tf.keras.losses.Loss):
    """
    Weighted Mean Squared Error Loss Function for tensorflow neural network
    
    Usage:
        - Define list of weights with len = labels
        - Use weights as arguments - no need to square, this is handled in-function
        - Typical usage - defining target precision on outputs for the network to achieve, weights parameters in loss calculation to force network to focus on parameters with unc >> weight.
    
    """
    
    def __init__(self, weights, name = "WMSE",**kwargs):
        super(WMSE, self).__init__()
        self.weights = np.float32(weights)
        
    def call(self, y_true, y_pred):
        loss = ((y_true - y_pred)/(self.weights))**2
        return tf.math.reduce_mean(loss)
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'weights': self.weights
        })
        return config

def WMSE_metric(y_true, y_pred):
    metric = ((y_true - y_pred)/(weights))**2
    return tf.reduce_mean(metric)


class emulator:
    def __init__(self, emulator_name, file_path='pitchfork/'):
        self.emulator_name = emulator_name
        self.file_path = file_path + self.emulator_name
        
        with open(self.file_path+".pkl", 'rb') as fp:
             self.emulator_dict = pickle.load(fp)

        self.log_inputs_mean = np.array(self.emulator_dict["data_scaling"]["inp_mean"][0])
        
        self.log_inputs_std = np.array(self.emulator_dict["data_scaling"]["inp_std"][0])

        self.log_outputs_mean = np.array(self.emulator_dict["data_scaling"]["classical_out_mean"][0] + self.emulator_dict["data_scaling"]["astero_out_mean"][0])
        
        self.log_outputs_std = np.array(self.emulator_dict["data_scaling"]["classical_out_std"][0] + self.emulator_dict["data_scaling"]["astero_out_std"][0])
            
        self.custom_objects = {"InversePCA": InversePCA(self.emulator_dict['custom_objects']['inverse_pca']['pca_comps'], self.emulator_dict['custom_objects']['inverse_pca']['pca_mean']),"WMSE": WMSE(self.emulator_dict['custom_objects']['WMSE']['weights'])}

        self.model = tf.keras.models.load_model(self.file_path+".h5", custom_objects=self.custom_objects)

        [print(str(key).replace("log_","") + " range: " + "[min = " + str(self.emulator_dict['parameter_ranges'][key]["min"]) + ", max = " + str(self.emulator_dict['parameter_ranges'][key]["max"]) + "]") for key in self.emulator_dict['parameter_ranges'].keys()];

    def predict(self, input_data, n_min=6, n_max=40, verbose=False):
        
        log_inputs = np.log10(input_data)
        
        standardised_log_inputs = (log_inputs - self.log_inputs_mean)/self.log_inputs_std

        standardised_log_outputs = self.model(standardised_log_inputs)

        standardised_log_outputs = np.concatenate((np.array(standardised_log_outputs[0]),np.array(standardised_log_outputs[1])), axis=1)

        log_outputs = (standardised_log_outputs*self.log_outputs_std) + self.log_outputs_mean

        outputs = 10**log_outputs

        outputs[:,2] = log_outputs[:,2] ##we want star_feh in dex

        teff = np.array(((outputs[:,1]*astropy.constants.L_sun) / (4*np.pi*constants.sigma*((outputs[:,0]*astropy.constants.R_sun)**2)))**0.25)
        
        outputs[:,0] = teff
        
        outputs = np.concatenate((np.array(outputs[:,:3]), np.array(outputs[:,n_min-3:n_max-2])), axis=1)

        return outputs